In [266]:
import numpy as np
import tensorflow as tf

In [267]:
%run 11-single_embedding_transformer.ipynb

  0%|          | 0/3 [00:00<?, ?it/s]

(64, 79, 512)
(64, 79, 512)
(64, 79, 512)
(64, 79, 512)
(64, 79, 512)
(64, 79, 512)
(64, 79)
(64, 79, 512)
(64, 79)
(64, 79, 11913)


In [268]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [269]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [270]:
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [271]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [272]:
print(tf.convert_to_tensor(input).shape)
print(main_labels.shape)

(64, 79)
(64, 79)


In [273]:
print(batched_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(64, 79), dtype=tf.int32, name=None), TensorSpec(shape=(64, 79), dtype=tf.int32, name=None))>


In [274]:
transformer.fit(batched_dataset,
                epochs=20)

Epoch 1/20
35/35 [==============================] - 120s 3s/step - loss: 9.3841 - masked_accuracy: 9.6067e-05
Epoch 2/20
35/35 [==============================] - 100s 3s/step - loss: 9.2950 - masked_accuracy: 0.0067
Epoch 3/20
35/35 [==============================] - 102s 3s/step - loss: 9.1439 - masked_accuracy: 0.0261
Epoch 4/20
35/35 [==============================] - 100s 3s/step - loss: 8.9481 - masked_accuracy: 0.0312
Epoch 5/20
35/35 [==============================] - 100s 3s/step - loss: 8.6967 - masked_accuracy: 0.0314
Epoch 6/20
35/35 [==============================] - 98s 3s/step - loss: 8.3959 - masked_accuracy: 0.0314
Epoch 7/20
35/35 [==============================] - 98s 3s/step - loss: 8.0659 - masked_accuracy: 0.0314
Epoch 8/20
35/35 [==============================] - 98s 3s/step - loss: 7.7322 - masked_accuracy: 0.0314
Epoch 9/20
35/35 [==============================] - 98s 3s/step - loss: 7.4219 - masked_accuracy: 0.0314
Epoch 10/20
35/35 [===========================

In [275]:
starting_string = "Romeo: he went"

In [276]:
used_tokenizer = data_object.tokenizer

In [277]:
encoded_input = used_tokenizer.texts_to_sequences([starting_string])[0]
encoded_input

[104, 20, 1301]

In [278]:
inner_token_list = [encoded_input]
output_generated = ""

In [281]:
print(inner_token_list)
print(output_generated)

[[104, 20, 1301, 2, 4, 23, 8, 91, 2, 4, 23, 8, 91, 2, 4, 23, 8, 91, 2, 4, 23, 8, 91, 2, 4, 23, 8, 91, 2, 1, 91, 2, 1, 91, 2, 4, 23, 8, 91, 2, 4, 23, 8, 91, 2, 1, 91, 2, 1, 91, 2, 1, 91, 5, 1, 91, 5, 1, 91, 5, 1, 91, 2, 1, 91, 2, 1, 91, 2, 1, 91, 2, 1, 91, 5, 1, 91, 5, 1, 91, 5, 1, 91, 5, 1, 91, 5, 1, 91, 2, 1, 91, 3, 1, 91, 2, 1, 91, 5, 1, 91, 2, 1]]
 and i have a man and i have a man and i have a man and i have a man and i have a man and the man and the man and i have a man and i have a man and the man and the man and the man of the man of the man of the man and the man and the man and the man and the man of the man of the man of the man of the man of the man and the man to the man and the man of the man and the


In [280]:
for i in range(100):
    inner_input = tf.convert_to_tensor(inner_token_list)
    predictions  = transformer(inner_input)[:, -1:, :]
    predicted_id = tf.argmax(predictions, axis=-1)
    token = predicted_id.numpy()[0][0]
    text = used_tokenizer.index_word[token]
    output_generated += (' ' + text)
    inner_token_list[0].append(token)

    

In [252]:
inner_input = tf.convert_to_tensor(inner_token_list) 
inner_input

<tf.Tensor: shape=(1, 27), dtype=int32, numpy=
array([[ 104,   20, 1301,    3,  587,    2,   67,   26,   18,    8,   91,
          13,    8,   91,    2,    8,   91,   11,    8,  309,    2,    8,
         309,   13,    8,  309,   13]])>

In [250]:
predictions  = transformer(inner_input)[:, -1:, :]
predicted_id = tf.argmax(predictions, axis=-1)
token = predicted_id.numpy()[0][0]
text = used_tokenizer.index_word[token]
output_generated += (' ' + text)
inner_token_list[0].append(token)




print(text)
print(token)

for
13


In [251]:
predictions

<tf.Tensor: shape=(1, 1, 11913), dtype=float32, numpy=
array([[[-14.466389 ,   3.3970423,   5.709853 , ..., -14.614429 ,
         -14.58085  , -14.690559 ]]], dtype=float32)>

In [265]:
transformer.save_weights('./transformer_v1/transformer_v1')

In [143]:
#inner_input = tf.transpose(working_input.stack())
inner_input = tf.convert_to_tensor(inner_token_list) 

predictions  = transformer(inner_input)[:, -1:, :]
predicted_id = tf.argmax(predictions, axis=-1)
token = predicted_id.numpy()[0][0]
text = used_tokenizer.index_word[token]
#working_input.concat(3)

#generated_o+=text
#output_array = working_input.write(my_iter_obj, predicted_id[0])

#my_iter_obj+=1

print(text)
print(predicted_id[0])

InvalidArgumentError: Exception encountered when calling layer 'positional_embedding_15' (type PositionalEmbedding).

{{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} slice index 1 of dimension 0 out of bounds. [Op:StridedSlice] name: transformer_4/decoder_9/positional_embedding_15/strided_slice/

Call arguments received by layer 'positional_embedding_15' (type PositionalEmbedding):
  • x=tf.Tensor(shape=(3,), dtype=int32)

In [286]:
for batch in batched_dataset:
    print(batch[0][0])
    break

tf.Tensor(
[  87  248  143   34  962  145  609  127   16  106   35  106  106   87
  248    7   41   35 1228  351    3  192   65    3 3678   35 1228 1228
   87  248   87    7   92 1052  214   11 2085  544    3    1  281   35
   34 2333   34 2333   87  248   67   78  474   26    2  369   23 1308
   53   37  162 2086  646    8 3679   35   33   50 4648 1013   67   15
   18  167  154  154  149  248   72  226   46], shape=(79,), dtype=int32)


In [130]:
tensor_encoded_input = input_to_tensor = tf.convert_to_tensor(encoded_input) 

In [131]:
tensor_encoded_input

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 104,   20, 1301])>

In [132]:
generated_o = ""

In [ ]:
#inner_input = tf.transpose(working_input.stack())


predictions  = transformer(inner_input)[:, -1:, :]
predicted_id = tf.argmax(predictions, axis=-1)
token = predicted_id.numpy()[0][0]
text = used_tokenizer.index_word[token]
working_input.concat(3)

generated_o+=text
output_array = working_input.write(my_iter_obj, predicted_id[0])

my_iter_obj+=1

print(text)
print(predicted_id[0])

In [134]:
working_input = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
working_input.write(0,tf.cast(tensor_encoded_input, dtype=tf.int64))

In [135]:
my_iter_obj = 1

In [136]:
print(generated_o)
print(working_input)

In [137]:
inner_input = working_input.stack()#tf.transpose(working_input.stack())
inner_input

<tf.Tensor: shape=(1, 3), dtype=int64, numpy=array([[ 104,   20, 1301]], dtype=int64)>

In [140]:
#inner_input = tf.transpose(working_input.stack())


predictions  = transformer(inner_input)[:, -1:, :]
predicted_id = tf.argmax(predictions, axis=-1)
token = predicted_id.numpy()[0][0]
text = used_tokenizer.index_word[token]
working_input.concat(3)

generated_o+=text
output_array = working_input.write(my_iter_obj, predicted_id[0])

my_iter_obj+=1

print(text)
print(predicted_id[0])

ValueError: Incompatible shape for value ((1,)), expected ((3,))

In [73]:
predictions  = transformer(tensor_encoded_input)[:, -1:, :]
predictions

<tf.Tensor: shape=(1, 1, 11913), dtype=float32, numpy=
array([[[-13.653607 ,   3.5102906,   4.55223  , ..., -13.595964 ,
         -13.772838 , -13.836386 ]]], dtype=float32)>

In [72]:
predicted_id = tf.argmax(predictions, axis=-1)
predicted_id

<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[3]], dtype=int64)>

3

In [84]:
text = used_tokenizer.index_word[predicted_id.numpy()]
#.lookup(predicted_id)[0]

TypeError: unhashable type: 'numpy.ndarray'

In [82]:
text[3]

'to'